# データ読込

In [1]:
import pandas as pd
from logic.factory_manage.sql import load_data_from_sqlite
from utils.get_holydays import get_japanese_holidays
from logic.factory_manage.predict_model_v3 import generate_features, train_and_predict

from utils.font import set_jp_font
set_jp_font()

# CSVファイル読み込み（パスは適宜変更）
path = "/work/app/data/factory_manage/weight_data.db"
df = load_data_from_sqlite()
df["伝票日付"].max()

hol_max = df["伝票日付"].max()
hol_min = df["伝票日付"].min()
print(f"最小日付: {hol_min}, 最大日付: {hol_max}")
holiday = get_japanese_holidays(hol_min, hol_max)
df.head()

✅ 日本語フォントを設定しました: Noto Sans CJK JP
最小日付: 2024-05-01 00:00:00, 最大日付: 2025-06-25 00:00:00


,伝票日付,品名,正味重量,祝日フラグ
0,2024-05-01,混合廃棄物A,1620.0,0
1,2024-05-01,混合廃棄物A,1840.0,0
2,2024-05-01,混合廃棄物A,1730.0,0
3,2024-05-01,混合廃棄物B,2000.0,0
4,2024-05-01,混合廃棄物A,360.0,0


In [2]:
# 品目別合計重量を集計
item_totals = df.groupby("品名")["正味重量"].sum().sort_values(ascending=False)

print(item_totals)


品名
混合廃棄物A          15072470.0
混合廃棄物B           3896550.0
GC 軽鉄･ｽﾁｰﾙ類      2463090.0
選別               1536300.0
木くず              1412820.0
                   ...    
混合廃棄物A(廃ﾌﾟﾗ)          30.0
雑誌                    30.0
選別（水銀灯）               30.0
ﾀｲﾔ                   20.0
混合廃棄物(羽毛)             10.0
Name: 正味重量, Length: 156, dtype: float64


In [3]:
# pdは既にCELL INDEX:1でimportされているので、そのまま使えます
df_2021 = pd.read_csv("/work/app/data/input/2021顧客.csv", encoding="utf-8")
df_2022 = pd.read_csv("/work/app/data/input/2022顧客.csv", encoding="utf-8")
df_2023 = pd.read_csv("/work/app/data/input/2023_all.csv", encoding="utf-8")
df_2024 = pd.read_csv("/work/app/data/input/20240501-20250422.csv", encoding="utf-8")

df_2021 = df_2021[['伝票日付', '商品', '正味重量']]
df_2022 = df_2022[['伝票日付', '商品', '正味重量']]
df_2023 = df_2023[['伝票日付', '商品', '正味重量']]
df_2021.rename(columns={'商品': '品名'}, inplace=True)
df_2022.rename(columns={'商品': '品名'}, inplace=True)
df_2023.rename(columns={'商品': '品名'}, inplace=True)
df_2024 = df_2024[['伝票日付', '品名', '正味重量']]

df_all = pd.concat([df_2021, df_2022, df_2023, df_2024], ignore_index=True)
# 曜日など () を削除
df_all["伝票日付"] = df_all["伝票日付"].str.replace(r"\(.*\)", "", regex=True)
df_all["伝票日付"] = pd.to_datetime(df_all["伝票日付"], format="%Y/%m/%d")
df_all


/tmp/ipykernel_1647759/4147401483.py:4: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv("/work/app/data/input/2023_all.csv", encoding="utf-8")


,伝票日付,品名,正味重量
0,2021-01-04,室内機,1440.0
1,2021-01-04,混合廃棄物A,1440.0
2,2021-01-04,室外機,1440.0
3,2021-01-05,混合廃棄物A,920.0
4,2021-01-05,混合廃棄物A,870.0
...,...,...,...
183774,2025-05-26,軽量物系A(ｽﾀｲﾛﾌｫｰﾑ),10.0
183775,2025-05-26,廃ﾌﾟﾗｽﾁｯｸ類,30.0
183776,2025-05-26,廃ﾌﾟﾗｽﾁｯｸ類,160.0
183777,2025-05-26,混合廃棄物A,2530.0


In [4]:
df_reserve = pd.read_csv(
    "/work/app/data/input/yoyaku_data.csv")
df_reserve["予約日"] = pd.to_datetime(df_reserve["予約日"])
df_reserve = df_reserve[df_reserve["予約日"] >= hol_min]
df_reserve = df_reserve[df_reserve["予約日"] <= hol_max]
print(df_reserve["予約日"].min(), df_reserve["予約日"].max())

2024-05-01 00:00:00 2025-05-31 00:00:00


In [5]:
print(df_reserve.columns)

Index(['予約日', '予約得意先名', '固定客', '台数'], dtype='object')


### 受入番号用

In [6]:
import os
import glob
import pandas as pd

# ディレクトリ内の全CSVファイルパスを取得
csv_dir = "/work/app/data/input/受入_時刻"
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

# 全CSVを読み込んで結合
dfs = []
for f in csv_files:
    df_tmp = pd.read_csv(f)

    # 伝票日付を整形 → 日付型へ変換
    df_tmp["伝票日付"] = df_tmp["伝票日付"].str.replace(r"\(.*?\)", "", regex=True).str.strip()
    df_tmp["伝票日付"] = pd.to_datetime(df_tmp["伝票日付"], format="%Y/%m/%d")

    # 正味重量をカンマ除去して数値化
    df_tmp["正味重量"] = df_tmp["正味重量"].replace({',': ''}, regex=True).astype(float)

    # 受入番号の欠損を埋めて型変換（念のため）
    df_tmp["受入番号"] = df_tmp["受入番号"].fillna(-1).astype(int)

    dfs.append(df_tmp)

df_Ukeire = pd.concat(dfs, ignore_index=True)

# 必要カラムだけ抽出（品名・重量・受入番号）
df_Ukeire = df_Ukeire[["伝票日付", "品名", "正味重量", "受入番号"]].copy()

# 台数カウント準備（日付・品名ごとの受入番号ユニーク数）
df_count = (
    df_Ukeire.groupby(["伝票日付", "品名"])["受入番号"]
    .nunique()
    .reset_index()
    .rename(columns={"受入番号": "台数"})
)

# （参考）結合しておく場合
df_merged = pd.merge(df_Ukeire, df_count, on=["伝票日付", "品名"], how="left")

# 結果確認
df_merged


,伝票日付,品名,正味重量,受入番号,台数
0,2025-06-01,混合廃棄物A,1110.0,54050,35
1,2025-06-01,混合廃棄物A,120.0,54094,35
2,2025-06-01,混合廃棄物A,270.0,54060,35
3,2025-06-01,混合廃棄物A,1220.0,54044,35
4,2025-06-01,混合廃棄物A,1650.0,54089,35
...,...,...,...,...,...
64680,2024-11-30,混合廃棄物A,50.0,31824,74
64681,2024-11-30,混合廃棄物A,310.0,31822,74
64682,2024-11-30,混合廃棄物A,1140.0,31805,74
64683,2024-11-30,木くず,870.0,31824,9


In [7]:
target_items = ["混合廃棄物A", "混合廃棄物B", "GC 軽鉄･ｽﾁｰﾙ類", "選別", "木くず"]

# モデル作成・失敗


In [ ]:
# # 最初のモデル
from logic.factory_manage.predict_model_v4_4 import full_walkforward_pipeline

all_actual, all_pred  = full_walkforward_pipeline(
    df
)

# print_metrics(all_actual, all_pred)

In [ ]:
# # 最初のモデル・修正・未来リークなし
from logic.factory_manage.predict_model_v4_4_fix1 import full_walkforward_pipeline

all_actual, all_pred  = full_walkforward_pipeline(
    df
)

# print_metrics(all_actual, all_pred)

In [ ]:
# 交差検証パイプライン
from logic.factory_manage.predict_model_v4_5 import cross_validation_pipeline
    
pred_df_slim = cross_validation_pipeline(
    df,
    n_splits=5,)


In [ ]:
## 最適履歴データ数
from logic.factory_manage.predict_model_v4_5 import prepare_history_data,full_walkforward_pipeline,print_metrics

window_list = [180,240,360]

for history_days in window_list:
    df_input = prepare_history_data(df, history_days=history_days)
    if df_input is not None:
        all_actual, all_pred = full_walkforward_pipeline(df_input, start_index=30)
        print(f"\n履歴日数: {history_days}日")
        print_metrics(all_actual, all_pred)


# 真モデル

In [ ]:
# # 最初のモデル
from logic.factory_manage.predict_model_v4_2_1 import full_walkforward

hol_max_all = df["伝票日付"].max()
hol_min_all = df["伝票日付"].min()
print(f"最小日付: {hol_min_all}, 最大日付: {hol_max_all}")
holiday_all = get_japanese_holidays(hol_min_all, hol_max_all)


full_walkforward(
    df_all,
    holidays=holiday_all,
    top_n=2
)

# print_metrics(all_actual, all_pred)

In [19]:
from logic.factory_manage.predict_model_v4_2_2 import generate_reserve_features, full_walkforward
# from logic.factory_manage.holiday_jp import get_japanese_holidays  # 必要に応じて追加
import pandas as pd

# ---------- 予約特徴量生成 ----------
df_reserve_feat = generate_reserve_features(df_reserve)

# 対象日：予約日と一致する日だけ残す
reserve_dates = df_reserve_feat.index
df_all["伝票日付"] = pd.to_datetime(df_all["伝票日付"])  # 念のため変換
valid_dates = df_all["伝票日付"].isin(reserve_dates)
df_all = df_all[valid_dates].copy()

# ---------- 最新から〇日分に制限 ----------
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt

# 評価対象のdaysリスト
days_list = [330]
results = []

for days in days_list:
    print(f"\n=== {days}日分のデータで評価中 ===")
    
    latest_date = df_all["伝票日付"].max()
    cutoff_date = latest_date - pd.Timedelta(days=days)
    df_subset = df_all[df_all["伝票日付"] >= cutoff_date].copy()
    
    hol_max = df_subset["伝票日付"].max()
    hol_min = df_subset["伝票日付"].min()
    holidays = get_japanese_holidays(hol_min, hol_max)
    
    try:
        actual, pred = full_walkforward(
            df_subset,
            df_reserve=df_reserve,
            holidays=holidays,
            top_n=2
        )
        if actual and pred:
            r2 = r2_score(actual, pred)
            mae = mean_absolute_error(actual, pred)
            results.append((days, r2, mae))
            print(f"✅ R² = {r2:.3f}, MAE = {mae:,.0f}kg")
        else:
            print("⚠ 評価に十分なデータがありません")
    except Exception as e:
        print(f"❌ エラー: {e}")
        results.append((days, None, None))

# 結果表示
df_result = pd.DataFrame(results, columns=["days", "R2", "MAE"])
print("\n=== 評価結果 ===")
print(df_result)

# 可視化（任意）
plt.plot(df_result["days"], df_result["R2"], marker="o")
plt.xlabel("Days")
plt.ylabel("R² Score")
plt.title("日数別 R² 評価")
plt.grid(True)
plt.show()




=== 330日分のデータで評価中 ===
▶️ full_walkforward 開始
📌 入力データ件数: 51768

=== 2024-08-10 を予測中 ===
▶️ train_and_predict_stage1 開始
📌 df_feat_today index: DatetimeIndex(['2024-08-10'], dtype='datetime64[ns]', name='伝票日付', freq=None)
📌 学習用特徴量サイズ: (30, 21)
📌 学習用pivotサイズ: (30, 244)
🛠 モデル訓練中: elastic for 混合廃棄物A
🛠 モデル訓練中: rf for 混合廃棄物A
✅ 混合廃棄物A 予測: 41581.9kg / 正解: 36970.0kg
🔍 ElasticNet 係数 (混合廃棄物A):
   混合廃棄物A_前日値                : 669.2100
   混合廃棄物B_前日値                : 627.8468
   混合廃棄物A_前週平均               : -2567.5977
   混合廃棄物B_前週平均               : 1349.3739
   合計_前日値                    : 707.9577
   合計_3日平均                   : 3600.6179
   合計_前週平均                   : -1656.0330
   曜日                        : -1168.2647
   週番号                       : 903.3811
   1台あたり重量_過去中央値             : 1866.8366
   祝日フラグ                     : -1500.6168
   祝日後フラグ                    : 1492.9354
   予約件数                      : 1117.9995
   合計台数                      : 2643.4056
   固定客予約数                    : 3849.5428


KeyboardInterrupt: 

In [28]:
from logic.factory_manage.predict_model_v4_2_2 import generate_reserve_features, full_walkforward
# from logic.factory_manage.holiday_jp import get_japanese_holidays  # 必要に応じて追加
import pandas as pd

# ---------- 予約特徴量生成 ----------
df_reserve_feat = generate_reserve_features(df_reserve)

# 対象日：予約日と一致する日だけ残す
reserve_dates = df_reserve_feat.index
df_all["伝票日付"] = pd.to_datetime(df_all["伝票日付"])  # 念のため変換
valid_dates = df_all["伝票日付"].isin(reserve_dates)
df_all = df_all[valid_dates].copy()

# ---------- 最新から〇日分に制限 ----------
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt

# 評価対象のdaysリスト
days_list = [90, 120, 150, 180, 210, 240, 270, 300, 330, 360]
results = []

for days in days_list:
    print(f"\n=== {days}日分のデータで評価中 ===")
    
    latest_date = df_all["伝票日付"].max()
    cutoff_date = latest_date - pd.Timedelta(days=days)
    df_subset = df_all[df_all["伝票日付"] >= cutoff_date].copy()
    
    hol_max = df_subset["伝票日付"].max()
    hol_min = df_subset["伝票日付"].min()
    holidays = get_japanese_holidays(hol_min, hol_max)
    
    try:
        actual, pred = full_walkforward(
            df_subset,
            df_reserve=df_reserve,
            holidays=holidays,
            top_n=2
        )
        if actual and pred:
            r2 = r2_score(actual, pred)
            mae = mean_absolute_error(actual, pred)
            results.append((days, r2, mae))
            print(f"✅ R² = {r2:.3f}, MAE = {mae:,.0f}kg")
        else:
            print("⚠ 評価に十分なデータがありません")
    except Exception as e:
        print(f"❌ エラー: {e}")
        results.append((days, None, None))

# 結果表示
df_result = pd.DataFrame(results, columns=["days", "R2", "MAE"])
print("\n=== 評価結果 ===")
print(df_result)

# 可視化（任意）
plt.plot(df_result["days"], df_result["R2"], marker="o")
plt.xlabel("Days")
plt.ylabel("R² Score")
plt.title("日数別 R² 評価")
plt.grid(True)
plt.show()




=== 90日分のデータで評価中 ===
▶️ full_walkforward 開始
📌 入力データ件数: 14968

=== 2025-04-07 を予測中 ===
▶️ train_and_predict_stage1 開始
📌 df_feat_today index: DatetimeIndex(['2025-04-07'], dtype='datetime64[ns]', name='伝票日付', freq=None)
📌 学習用特徴量サイズ: (30, 21)
📌 学習用pivotサイズ: (30, 152)
🛠 モデル訓練中: elastic for 混合廃棄物A
🛠 モデル訓練中: rf for 混合廃棄物A
✅ 混合廃棄物A 予測: 45565.4kg / 正解: 47340.0kg
🔍 ElasticNet 係数 (混合廃棄物A):
   混合廃棄物A_前日値                : 2731.3394
   混合廃棄物B_前日値                : -2620.8090
   混合廃棄物A_前週平均               : 198.0885
   混合廃棄物B_前週平均               : -2113.3349
   合計_前日値                    : 1484.9743
   合計_3日平均                   : 379.9620
   合計_前週平均                   : -141.3546
   曜日                        : -1665.1407
   週番号                       : 4420.9272
   1台あたり重量_過去中央値             : 3605.6317
   祝日フラグ                     : -2169.6699
   祝日前フラグ                    : -761.6685
   祝日後フラグ                    : -1552.9051
   予約件数                      : 7391.4472
   合計台数                      : 3157.888

KeyboardInterrupt: 